In [1]:
import turicreate as tc

In [2]:
doc1 = tc.SFrame('key values age.csv')

Finished parsing file /root/notebooks/ML_GEO data/metadata-unsupervised-assessment/key values age.csv

Parsing completed. Parsed 100 lines in 0.020463 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /root/notebooks/ML_GEO data/metadata-unsupervised-assessment/key values age.csv

Parsing completed. Parsed 295 lines in 0.026579 secs.

In [3]:
[i for i in doc1]#

[{'key': 'age (1#pfu_1)', 'values': '21'},
 {'key': 'age (1#pvu_1)', 'values': '26'},
 {'key': 'sampling age', 'values': '38'},
 {'key': 'gestational age at delivery', 'values': '40'},
 {'key': 'age (1#pfu_2)', 'values': '42'},
 {'key': 'average age (days after birth)', 'values': '47'},
 {'key': 'age at rc', 'values': '51.33'},
 {'key': 'cell line source age', 'values': '56'},
 {'key': 'age (at specimen collection)', 'values': '67.73'},
 {'key': 'age_dx', 'values': '68'},
 {'key': 'sac age (wks)', 'values': '12/16/17'},
 {'key': 'age of diagnosis (year)', 'values': '"--", 34, 57, 32, 35'},
 {'key': 'age at onset (years)', 'values': '"-", na, 47, 48, 78'},
 {'key': 'age of onset', 'values': '"=-n/a", 20yr, -, 17 yr, 25 yr'},
 {'key': 'age at diagnosis (months)', 'values': '<18m, 1, >18m, 6, 4'},
 {'key': 'age of trees', 'values': '>30, 3, 9, 2, 1'},
 {'key': 'infant age', 'values': '0 day'},
 {'key': 'ageing at 29 c in days', 'values': '2000, 2, 14'},
 {'key': 'culture age (days)', 'val

In [114]:
keys = ['age', 'tissue', 'cell line']

In [19]:
# Remove stopwords and convert to bag of words
doc = tc.text_analytics.count_words(doc1['key'])
doc = doc.dict_trim_by_keys(tc.text_analytics.stopwords(), exclude=True)

IndexError: Invalid type to use for indexing

In [6]:
# Remove stopwords and convert to bag of words
doc2 = tc.text_analytics.count_words(doc1['values'])
doc2 = doc2.dict_trim_by_keys(tc.text_analytics.stopwords(), exclude=True)

In [7]:
# Learn topic model
model = tc.topic_model.create(doc)

Learning a topic model

Number of documents       295

Vocabulary size       271

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 15.389ms      | 475559         | 0               |

+-----------+---------------+----------------+-----------------+

In [8]:
model1 = tc.topic_model.create(doc2)

Learning a topic model

Number of documents       295

Vocabulary size       497

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 19.638ms      | 1.32539e+06    | 0               |

+-----------+---------------+----------------+-----------------+

model1 = tc.topic_model.create(doc2)

In [117]:
out = (model.get_topics())

In [118]:
out

topic,word,score
0,age,0.330669330669
0,blood,0.0509490509491
0,(days),0.030969030969
0,draw,0.020979020979
0,postnatal,0.020979020979
1,age,0.291949563531
1,start,0.0494665373424
1,tissue,0.0300678952473
1,source,0.0300678952473
1,collection,0.0300678952473


In [9]:
model

Class                          : TopicModel

Schema
------
Vocabulary Size                : 271

Settings
--------
Number of Topics               : 10
alpha                          : 5.0
beta                           : 0.1
Iterations                     : 10
Training time                  : 1.0157
Verbose                        : True

Accessible fields             : 
m.topics                      : An SFrame containing the topics.
m.vocabulary                  : An SArray containing the words in the vocabulary.
Useful methods                : 
m.get_topics()                : Get the most probable words per topic.
m.predict(new_docs)           : Make predictions for new documents.

In [10]:
model1

Class                          : TopicModel

Schema
------
Vocabulary Size                : 497

Settings
--------
Number of Topics               : 10
alpha                          : 5.0
beta                           : 0.1
Iterations                     : 10
Training time                  : 0.0197
Verbose                        : True

Accessible fields             : 
m.topics                      : An SFrame containing the topics.
m.vocabulary                  : An SArray containing the words in the vocabulary.
Useful methods                : 
m.get_topics()                : Get the most probable words per topic.
m.predict(new_docs)           : Make predictions for new documents.

In [11]:
table = model.get_topics()

In [12]:
table

topic,word,score
0,age,0.471511147812
0,patient,0.0338563170933
0,(months),0.0255986787779
0,death,0.0173410404624
0,months,0.0173410404624
1,age,0.358569926393
1,start,0.0536277602524
1,(y),0.0220820189274
1,source,0.0220820189274
1,draw,0.0220820189274


In [13]:
table.print_rows(50,3)

+-------+-------------+-----------------+
| topic |     word    |      score      |
+-------+-------------+-----------------+
|   0   |     age     |  0.471511147812 |
|   0   |   patient   | 0.0338563170933 |
|   0   |   (months)  | 0.0255986787779 |
|   0   |    death    | 0.0173410404624 |
|   0   |    months   | 0.0173410404624 |
|   1   |     age     |  0.358569926393 |
|   1   |    start    | 0.0536277602524 |
|   1   |     (y)     | 0.0220820189274 |
|   1   |    source   | 0.0220820189274 |
|   1   |     draw    | 0.0220820189274 |
|   2   |     age     |  0.312567132116 |
|   2   |    sample   | 0.0440386680988 |
|   2   |    onset    | 0.0332975295381 |
|   2   |  commenced  | 0.0225563909774 |
|   2   |    (yrs)    | 0.0225563909774 |
|   3   |     age     |  0.189789123196 |
|   3   |    blood    | 0.0566037735849 |
|   3   |    (days)   | 0.0344062153163 |
|   3   |  collection |  0.023307436182 |
|   3   |   culture   |  0.023307436182 |
|   4   |     age     |  0.3844601

In [15]:
table1 = model1.get_topics()

In [16]:
table1

topic,word,score
0,"years,",0.0844257996854
0,"days,",0.0791819611956
0,"68,",0.0372312532774
0,"48,",0.0319874147876
0,25,0.0319874147876
1,days,0.0702541106129
1,2,0.0602889885401
1,"60,",0.0303936223219
1,"61,",0.0303936223219
1,"51,",0.0254110612855


In [18]:
table1.print_rows(50,3)

+-------+--------+-----------------+
| topic |  word  |      score      |
+-------+--------+-----------------+
|   0   | years, | 0.0844257996854 |
|   0   | days,  | 0.0791819611956 |
|   0   |  68,   | 0.0372312532774 |
|   0   |  48,   | 0.0319874147876 |
|   0   |   25   | 0.0319874147876 |
|   1   |  days  | 0.0702541106129 |
|   1   |   2    | 0.0602889885401 |
|   1   |  60,   | 0.0303936223219 |
|   1   |  61,   | 0.0303936223219 |
|   1   |  51,   | 0.0254110612855 |
|   2   |  76,   | 0.0544918998527 |
|   2   |  day   | 0.0348551791851 |
|   2   |  53,   | 0.0348551791851 |
|   2   |   8,   | 0.0299459990182 |
|   2   | hours, | 0.0299459990182 |
|   3   | weeks  | 0.0835803200948 |
|   3   |   1,   | 0.0480142264375 |
|   3   |   7    | 0.0480142264375 |
|   3   |  65,   | 0.0302311796088 |
|   3   |   17   | 0.0302311796088 |
|   4   |  42,   | 0.0404777704048 |
|   4   |   40   | 0.0272063702721 |
|   4   |  58,   | 0.0272063702721 |
|   4   |  14,   | 0.0272063702721 |
|

In [20]:
topics = (model.get_topics(output_type='topic_words'))

In [22]:
topics.print_rows(10,1)

+-------------------------------+
|             words             |
+-------------------------------+
|  [weeks,, weeks, 1,, 5, na,]  |
|   [weeks, na,, 5,, 52,, na]   |
|  [years, days, 24,, 66,, 46,] |
|   [day, days,, 75,, 30, 26]   |
| [years,, old,, days, 48,, 6,] |
| [2, 27,, 29,, 20, sexfemale,] |
|    [57,, 54,, 63,, 19, 44,]   |
|  [76,, 61,, hours,, 40,, 50]  |
|     [12,, 7, 10, 58,, 60,]    |
|   [65,, week, 53,, 49,, 68,]  |
+-------------------------------+
[10 rows x 1 columns]



In [20]:
list_words = table1.groupby(key_column_names='word',operations={'sum_scores': tc.aggregate.SUM('score')})

In [21]:
list_words

word,sum_scores
"48,",0.0319874147876
"days,",0.0791819611956
"42,",0.0404777704048
3,0.045341305431
"28,",0.0216130732736
day,0.105109289798
"5,",0.0271710175812
"58,",0.0272063702721
weeks,0.0835803200948
"53,",0.0348551791851


In [64]:
list_words.print_rows(45,2)

+--------+-----------------+
|  word  |    sum_scores   |
+--------+-----------------+
| days,  |  0.123993558776 |
|  40,   | 0.0416911332942 |
|   3    | 0.0530339225991 |
|  28,   | 0.0550514216576 |
|  32,   |  0.037309511298 |
|  70,   | 0.0580654801826 |
|  day   |  0.10798236259  |
|  58,   | 0.0317212688508 |
|  25,   | 0.0235226620769 |
|  53,   | 0.0369026013309 |
|  60,   | 0.0267997898056 |
|  30,   | 0.0429522081065 |
|  45,   | 0.0235226620769 |
|   1    | 0.0292598967298 |
|  68,   | 0.0393788130893 |
|  54,   | 0.0551009274414 |
|  61,   | 0.0338325013866 |
|   5    | 0.0299471520846 |
|  49,   | 0.0482560917344 |
|  67,   | 0.0595426776209 |
|  week  | 0.0434782608696 |
|  23,   | 0.0369026013309 |
|  78,   | 0.0349971313827 |
|  26,   | 0.0496453900709 |
|  56,   | 0.0522088353414 |
|  10,   | 0.0299471520846 |
| years  | 0.0782029950083 |
|  22,   | 0.0369214768591 |
|   8    | 0.0320546505518 |
|  72,   | 0.0421216848674 |
|  39,   | 0.0530740935365 |
| hours, | 0.0

In [22]:
topics_features = list_words.sort('sum_scores', ascending= False)

In [23]:
topics_features.print_rows(45,2)

+--------+-----------------+
|  word  |    sum_scores   |
+--------+-----------------+
| years  |  0.127042698998 |
| weeks, |  0.124419204956 |
|  day   |  0.105109289798 |
| years, | 0.0844257996854 |
| weeks  | 0.0835803200948 |
| days,  | 0.0791819611956 |
|  65,   | 0.0720484227683 |
|  days  | 0.0702541106129 |
|  na,   |  0.067630356221 |
|   2    | 0.0602889885401 |
|  week  | 0.0546832701678 |
|  76,   | 0.0544918998527 |
|   1,   | 0.0480142264375 |
|   7    | 0.0480142264375 |
|   3    |  0.045341305431 |
|   30   | 0.0426989984186 |
|  old,  | 0.0426989984186 |
|  39,   | 0.0418172431595 |
|  42,   | 0.0404777704048 |
|  49,   | 0.0384407146724 |
|  68,   | 0.0372312532774 |
|  52,   | 0.0366546205472 |
|  53,   | 0.0348551791851 |
|  12,   | 0.0330265295073 |
|   3,   | 0.0324986680874 |
|  63,   | 0.0321560358461 |
|   25   | 0.0319874147876 |
|  48,   | 0.0319874147876 |
|  60,   | 0.0303936223219 |
|  61,   | 0.0303936223219 |
|   17   | 0.0302311796088 |
| hours, | 0.0

In [16]:
#vector
all_features = [i for i in topics_features['word']]

#drop the keys
features = set(all_features) - set(keys)
features_nokeys = [k for k in features]

#include the scores

In [17]:
features_nokeys

['epilepsy',
 'sampling',
 'sample',
 'culture',
 '(range)',
 'birth)',
 '(years)',
 'maternal',
 'surgery',
 '(in',
 'gestational',
 'onset',
 'death',
 'group',
 'cell',
 'start',
 'experiment',
 'treatment',
 'day)',
 'donor',
 'draw',
 'patient',
 'ffpe',
 'time',
 '(yrs)',
 'diet',
 '(year)',
 'pig',
 '(months)',
 '(y)',
 '(days)',
 'days',
 'diagnosis',
 'withdrawal',
 '(yr)',
 'blood']

In [24]:
type(features)

set

In [15]:
topics_features.print_rows(37,2)

+-------------+-----------------+
|     word    |    sum_scores   |
+-------------+-----------------+
|     age     |  2.38472097976  |
|   (years)   |  0.16329733227  |
|  diagnosis  |  0.150684931507 |
|    death    | 0.0681520314548 |
|    (yrs)    | 0.0668414154653 |
|    start    | 0.0668127053669 |
|   culture   | 0.0636759537059 |
|    onset    |  0.059513830679 |
|    blood    | 0.0540882259973 |
|   patient   | 0.0537352555701 |
|   (months)  | 0.0537352555701 |
|     days    | 0.0449069003286 |
|  collection | 0.0449069003286 |
|     (yr)    |  0.040629095675 |
|    years    |  0.040629095675 |
|     cell    |  0.040629095675 |
|     (in     |  0.040629095675 |
|    group    | 0.0386052303861 |
|    sample   | 0.0343671416597 |
|   surgery   | 0.0339539978094 |
|    (days)   | 0.0339539978094 |
|     (at     | 0.0300096805421 |
|   sampling  | 0.0300096805421 |
|    tissue   | 0.0286241920591 |
| gestational | 0.0286241920591 |
|   current   | 0.0261519302615 |
|    days)    

In [7]:
out.print_rows(50,3)

+-------+-------------+-----------------+
| topic |     word    |      score      |
+-------+-------------+-----------------+
|   0   |     age     |  0.107947805457 |
|   0   |   patient   | 0.0486358244365 |
|   0   |     (at     | 0.0367734282325 |
|   0   |     diet    | 0.0367734282325 |
|   0   |   sampling  | 0.0249110320285 |
|   1   |     age     |  0.512154233026 |
|   1   |  collection | 0.0259849119866 |
|   1   |  operation  | 0.0176026823135 |
|   1   |  (postnatal | 0.0176026823135 |
|   1   |     (yr)    | 0.0176026823135 |
|   2   |   (years)   | 0.0668414154653 |
|   2   | gestational | 0.0668414154653 |
|   2   |  commenced  | 0.0275229357798 |
|   2   |     time    | 0.0275229357798 |
|   2   |   (weeks)   | 0.0275229357798 |
|   3   |    tissue   | 0.0492196878751 |
|   3   |     days    | 0.0492196878751 |
|   3   |   surgery   | 0.0372148859544 |
|   3   |    start    | 0.0372148859544 |
|   3   |   maternal  | 0.0372148859544 |
|   4   |     age     |  0.3887349

In [8]:
topics = (model.get_topics(output_type='topic_words'))

In [9]:
topics

words
"[age, patient, (at, diet,sampling] ..."
"[age, collection,operation, (yr), ..."
"[(years), gestational,commenced, (weeks), t ..."
"[tissue, days, start,surgery, maternal] ..."
"[age, (yrs), gestational,donor, start] ..."
"[age, cell, (days),group, culture] ..."
"[age, (yrs), culture,agenotype, sacrifice] ..."
"[age, diagnosis, onset,years, treatment] ..."
"[age, (months), (years),sample, biopsy] ..."
"[(years), death, blood,onset, draw] ..."


In [80]:
topics.export_csv('topics.csv')

In [81]:
model

Class                          : TopicModel

Schema
------
Vocabulary Size                : 273

Settings
--------
Number of Topics               : 10
alpha                          : 5.0
beta                           : 0.1
Iterations                     : 10
Training time                  : 0.0116
Verbose                        : True

Accessible fields             : 
m.topics                      : An SFrame containing the topics.
m.vocabulary                  : An SArray containing the words in the vocabulary.
Useful methods                : 
m.get_topics()                : Get the most probable words per topic.
m.predict(new_docs)           : Make predictions for new documents.

In [82]:
model.vocabulary

dtype: str
Rows: 273
['plants', 'age', 'biopsy', '(y)', 'diagnosis', 'diagnosis)', '(at', '(wks)', 'sac', '(1#pfu_1)', 'pool', 'taqman', 'op', 'duration', '(yr)', 'epilepsy', 'rat', '(years)', 'ffpe', 'block', 'mouse', 'examination', 'age-at-examination', '(1#pvu_1)', 'trees', 'primary', 'age-at-biopsy-years', 'tissue', '(1#pfu_2)', 'sampling', 'onset', '(wk)', 'mice', '(yr/mo)', 'ipi', "mother's", '(year)', 'maternal', '(yrs)', 'host', '65', '≥', 'gestation', 'age-at-onset', 'days)', '(postnatal', 'death', 'withdrawal', 'blood', 'vitro', 'years)', '(in', 'collection', 'age_at_diagnosis', 'group', '(month)', 'age@dx', '(pup)', 'average', 'age_diag', '(days)', 'harvested', 'birth)', '(after', 'sacrifice', 'description', 'category', '(months)', 'sample', 'commenced', 'high', 'salt', 'diet', 'culture', 'age/gender', 'diagonosis', 'age(yrs.)', 'seedlings', 'age.brain', 'clone', 'age.blood', 'drawing', 'gestation)', '(weeks', 'surgery', 'age(years)', 'unit', 'developmental', '(years', 'surg

In [83]:
pred = model.predict(doc)

In [84]:
for i in pred:
    print(i)

1
7
8
8
0
7
9
6
9
9
7
4
7
3
4
9
6
7
2
3
7
9
7
9
7
8
5
7
6
4
7
8
4
8
9
9
7
9
1
7
6
6
9
2
9
4
1
8
5
5
9
7
9
6
8
9
6
9
8
8
6
9
9
5
8
5
0
3
8
6
2
8
4
1
6
9
6
9
7
0
1
9
0
8
6
6
6
3
4
7
3
1
6
4
8
4
9
6
6
3
7
8
9
7
7
9
2
4
4
5
8
9
9
7
0
9
1
4
6
5
1
5
9
1
9
3
9
8
9
8
0
8
1
7
5
8
4
5
8
6
9
3
8
6
8
9
3
5
4
4
8
3
2
2
6
0
9
3
6
7
8
4
5
8
4
9
4
8
4
8
8
5
5
2
1
2
4
8
8
8
3
9
5
8
8
4
6
6
9
9
8
7
8
2
4
6
0
9
7
7
0
8
4
4
7
3
9
9
8
7
7
8
9
3
6
7
7
5
0
7
7
4
4
5
9
9
5
8
8
6
6
9
9
5
7
0
5
9
7
7
8
6
9
7
0
8
6
3
9
8
0
8
7
8
8
6
8
7
8
8
4
7
6
4
9
6
8
1
5
4
8
3
6
4
9
6
3
8
6
5
8
8
7
8
8
5
6
8
8
9
7
5
8
8
8
8
2


In [85]:
prob = model.predict(doc, output_type = 'probability')

In [86]:
for j in prob:
    print(j)

array('d', [0.11538461538461539, 0.09615384615384616, 0.09615384615384616, 0.09615384615384616, 0.09615384615384616, 0.09615384615384616, 0.09615384615384616, 0.09615384615384616, 0.11538461538461539, 0.09615384615384616])
array('d', [0.11538461538461539, 0.09615384615384616, 0.09615384615384616, 0.09615384615384616, 0.11538461538461539, 0.09615384615384616, 0.09615384615384616, 0.09615384615384616, 0.09615384615384616, 0.09615384615384616])
array('d', [0.09433962264150944, 0.11320754716981132, 0.11320754716981132, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.11320754716981132])
array('d', [0.11320754716981132, 0.11320754716981132, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.11320754716981132, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944])
array('d', [0.11320754716981132, 0.09433962264150944, 0.09433962264150944, 0.09433962264150944, 0.0943396226